In [1]:
import sys
sys.path.append('/mnt/c/Users/luukv/Documenten/NatuurSterrkenkundeMasterProject/CodeMP/MasterProject')

from functions import *
from import_data import *

from astropy.constants import R_sun, L_sun, sigma_sb, G, M_sun
import re
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import Markdown as md

In [2]:
# Import data
df_hmxb = HMXB_parameters()
df_falenga = falenga()
df_stellar_params = stellar_params()
df_photometric_params = photometric_params()
df_BJ = BailerJones()

<font color='yellow' size=5> Calculate observed luminosity from photometric filters </font>


In [3]:
df_L = pd.DataFrame({'id': df_hmxb['id'].copy(),
                              "L_true": [None] * len(df_hmxb['id']),
                              "L_true_err_low": [None] * len(df_hmxb['id']),
                              "L_true_err_high": [None] * len(df_hmxb['id']),
                              "ST": [None] * len(df_hmxb['id']),
                              "logL_true/L_sun": [None] * len(df_hmxb['id']),
                              "logL_true/L_sun_err_high": [None] * len(df_hmxb["id"]),
                              "logL_true/L_sun_err_low": [None] * len(df_hmxb["id"]),
                              "Teff": [None] * len(df_hmxb['id']),
                              "Teff_err": [None] * len(df_hmxb['id']),
                              "logTeff": [None] * len(df_hmxb['id']),
                              "logTeff_err": [None] * len(df_hmxb['id'])})

In [4]:
for i in range(df_hmxb.shape[0]):
    # Object id
    id = df_hmxb.loc[i, 'id']

    # Spectral type of object
    spectral_type = df_hmxb.loc[i, "ST"]

    # Effective temperature based on spectral type
    Teff = interpolate(df2=df_stellar_params, spectral_type=spectral_type, quantity='Teff')
    Teff_err = Teff_error(spectral_type)
    df_L.loc[df_L["id"] == df_hmxb.loc[i, 'id'], "Teff"] = Teff
    df_L.loc[df_L["id"] == df_hmxb.loc[i, 'id'], "Teff_err"] = Teff_err
    df_L.loc[df_L["id"] == df_hmxb.loc[i, 'id'], "logTeff"] = np.log10(Teff)
    df_L.loc[df_L["id"] == df_hmxb.loc[i, 'id'], "logTeff_err"] = Teff_err / (np.log(10) * Teff)



    if math.isnan(df_hmxb.loc[i, 'J']) == False and math.isnan(df_hmxb.loc[i, 'H']) == False and id in df_BJ['id'].tolist():
        # Distance
        distance = df_BJ.loc[df_BJ['id'] == id, 'r_med_photogeo'].reset_index(drop=True).at[0]
        distance_low = df_BJ.loc[df_BJ['id'] == id, 'r_lo_photogeo'].reset_index(drop=True).at[0]
        distance_high = df_BJ.loc[df_BJ['id'] == id, 'r_hi_photogeo'].reset_index(drop=True).at[0]
        Sigmad_low = distance - distance_low
        Sigmad_high = distance_high - distance

        # Expected (B-V)0 of object based on spectral type
        JH0 = interpolate(df2=df_photometric_params, spectral_type=spectral_type, quantity='(J-H)0')
        JH0_err = 0

        # Observed (B-V) of object based on simbad filters
        JHobs = df_hmxb.loc[i, 'J'] - df_hmxb.loc[i, 'H']
        JHobs_err = np.sqrt(0.03**2 + 0.03**2)

        # Bolometric correction (BC)
        BCh = interpolate(df2=df_photometric_params, spectral_type=spectral_type, quantity='BCh')

        # Calculate extinction
        Ah, Ah_err = extinction_and_error(0.46, 0.01, JHobs, JHobs_err, JH0, JH0_err)

        # Visual magnitude
        mh = df_hmxb.loc[i, 'H']

        # Calculate Absulute magnitude (visual)
        Mh = mh - 5 * np.log10(distance) + 5 - Ah

        # Calculate bolomatric absolute magnitude
        Mbol = Mh + BCh

        # Calculate the luminosity in solar luminosities
        L = 10**((Mbol - 4.74) / (-2.5))

        # Calculate the error on the luminosity
        L_err_low, L_err_high = luminosity_error_asymmetric(BCh, 0.1, mh, 0.03, distance, Sigmad_high, Sigmad_low, Ah, Ah_err)

        # Put L in luminosity dataframe
        df_L.loc[df_L["id"] == df_hmxb.loc[i, 'id'], "L_true"] = L
        df_L.loc[df_L["id"] == df_hmxb.loc[i, 'id'], "L_true_err_high"] = L_err_high
        df_L.loc[df_L["id"] == df_hmxb.loc[i, 'id'], "L_true_err_low"] = L_err_low
        df_L.loc[df_L["id"] == df_hmxb.loc[i, 'id'], "logL_true/L_sun"] = math.log10(L)
        df_L.loc[df_L["id"] == df_hmxb.loc[i, 'id'], "logL_true/L_sun_err_high"] = L_err_high / (np.log(10) * L)
        df_L.loc[df_L["id"] == df_hmxb.loc[i, 'id'], "logL_true/L_sun_err_low"] = L_err_low / (np.log(10) * L)
        df_L.loc[df_L["id"] == df_hmxb.loc[i, 'id'], "ST"] = spectral_type

    if id == 'Cen X-3':
        # Distance
        distance = df_BJ.loc[df_BJ['id'] == id, 'r_med_photogeo'].reset_index(drop=True).at[0]
        distance_low = df_BJ.loc[df_BJ['id'] == id, 'r_lo_photogeo'].reset_index(drop=True).at[0]
        distance_high = df_BJ.loc[df_BJ['id'] == id, 'r_hi_photogeo'].reset_index(drop=True).at[0]
        Sigmad_low = distance - distance_low
        Sigmad_high = distance_high - distance

        # Bolometric correction (BC)
        BCv = interpolate(df2=df_photometric_params, spectral_type=spectral_type, quantity='BCv')

        # Calculate extinction
        Av = 3.2 * 1.4

        # Calculate Absulute magnitude (visual)
        # V=12.27 From INTEGRAL V band
        Mv = 12.27 - 5 * np.log10(distance) + 5 - Av

        # Calculate bolomatric absolute magnitude
        Mbol = Mv + BCv

        # Calculate the luminosity in solar luminosities
        L = 10**((Mbol - 4.74) / (-2.5))

        # Calculate error on luminosity
        L_err_low, L_err_high = luminosity_error_asymmetric(BCv, 0.1, 12.27, 0.03, distance, Sigmad_high, Sigmad_low, Av, 0.02)

        # Put L in luminosity dataframe
        df_L.loc[df_L["id"] == df_hmxb.loc[i, 'id'], "L_true"] = L
        df_L.loc[df_L["id"] == df_hmxb.loc[i, 'id'], "L_true_err_high"] = L_err_high
        df_L.loc[df_L["id"] == df_hmxb.loc[i, 'id'], "L_true_err_low"] = L_err_low
        df_L.loc[df_L["id"] == df_hmxb.loc[i, 'id'], "logL_true/L_sun"] = math.log10(L)
        df_L.loc[df_L["id"] == df_hmxb.loc[i, 'id'], "logL_true/L_sun_err_high"] = L_err_high / (np.log(10) * L)
        df_L.loc[df_L["id"] == df_hmxb.loc[i, 'id'], "logL_true/L_sun_err_low"] = L_err_low / (np.log(10) * L)
        df_L.loc[df_L["id"] == df_hmxb.loc[i, 'id'], "ST"] = spectral_type

df_L = df_L.dropna(subset=['L_true']).reset_index(drop=True)

In [5]:
df_L

,id,L_true,L_true_err_low,L_true_err_high,ST,logL_true/L_sun,logL_true/L_sun_err_high,logL_true/L_sun_err_low,Teff,Teff_err,logTeff,logTeff_err
0,2S0114+650,190922.028745069,25433.8752191080,33079.1662729426,B1Ia,5.280856,0.0752458973578245,0.0578549878904937,27748.0,1446,4.443232,0.022632
1,SMC X-1,300229.794355759,48396.8794130719,29370.0275603489,B0Ib,5.477454,0.0424849270212388,0.0700080340644946,29558.0,1446,4.470675,0.021246
2,LMC X-4,74754.5786797090,14316.5275364422,14316.5275364422,O8III,4.873638,0.0831733523070536,0.0831733523070536,33179.0,1446,4.520863,0.018927
3,Vela X-1,396729.453662231,47522.5832573603,45754.9129785917,B0.5Ib,5.598494,0.0500872976360454,0.0520223428937318,28653.0,1446,4.45717,0.021917
4,Cen X-3,584693.803275,97415.1008700270,130729.428234661,O6.5III,5.766928,0.0971022251076722,0.0723572586624377,35895.0,1446,4.555034,0.017495
5,GX301-2,1044002.38712911,140816.631078772,148898.959954491,B1Ia+,6.018701,0.0619404682083080,0.0585783007698749,27748.0,1446,4.443232,0.022632
6,4U1538-52,81089.4616054329,15250.3098482039,13031.3209878923,B0Ia,4.908964,0.0697924327638216,0.0816767713493616,29558.0,1446,4.470675,0.021246
7,4U1700-37,501427.154383974,60194.6007196830,71433.1665275928,O6Iaf+,5.700208,0.0618694655376667,0.0521355548943988,36801.0,1446,4.56586,0.017064
8,LMC X-1,272650.185893583,52220.1625274968,52220.1625274968,O8III,5.435606,0.0831795817613509,0.0831795817613509,33179.0,1446,4.520863,0.018927
9,Cyg X-1,237897.865441185,28477.2847453852,28482.4105844587,O9.7Iab,5.376391,0.0519960686708698,0.0519867111946241,30101.0,1446,4.478581,0.020863


<font color='yellow' size=5>Calculate observed radius from observed luminosity and Teff from the models of Martins et al. (2005a) </font>

In [6]:
# Make radius dataframe
df_R = pd.DataFrame({'id': df_hmxb['id'].copy(),
                              "R_true": [None] * len(df_hmxb['id']),
                              "R_true_err": [None] * len(df_hmxb['id']),
                              "R_expected": [None] * len(df_hmxb['id']),
                              "R_expected_err_high": [None] * len(df_hmxb['id']),
                              "R_expected_err_low": [None] * len(df_hmxb['id']),
                              "ST": [None] * len(df_hmxb['id'])})

In [7]:
for i in range(df_L.shape[0]):
    # id
    id = df_L.loc[i, 'id']

    # Spectral type
    spectral_type = df_L.loc[i, 'ST']

    # True luminosity
    L = df_L.loc[df_L["id"] == id, "L_true"].reset_index(drop=True).at[0]
    L_err_high = df_L.loc[df_L["id"] == id, "L_true_err_high"].reset_index(drop=True).at[0]
    L_err_low = df_L.loc[df_L["id"] == id, "L_true_err_low"].reset_index(drop=True).at[0]
    
    # Effective temperature from model
    Teff = df_L.loc[df_L["id"] == id, "Teff"].reset_index(drop=True).at[0]
    Teff_err = df_L.loc[df_L["id"] == id, "Teff_err"].reset_index(drop=True).at[0]

    # Calculate the radius
    R, R_err_high, R_err_low = expected_radius_error_asymmetric(L, L_err_high, L_err_low, Teff, Teff_err)
    # R = math.sqrt((L_sun.value / R_sun.value**2) * (L / (4 * np.pi * sigma_sb.value * Teff**4)))
    # C = (4 * np.pi * sigma_sb.value * L_sun.value / R_sun.value**2)**(1/2)
    # R_err = ((C / ( 2 * L**(1/2) * Teff**2))**2 * L_err**2 + (2 * C * L**(1/2) / Teff**3)**2 * Teff_err**2)**(1/2)

    # Save luminosity in dataframe
    df_R.loc[df_R['id'] == id, "R_expected"] = R
    df_R.loc[df_R['id'] == id, "R_expected_err_high"] = R_err_high
    df_R.loc[df_R['id'] == id, "R_expected_err_low"] = R_err_low
    df_R.loc[df_R['id'] == id, "ST"] = spectral_type

<font color='yellow' size=5>Observed radius from Falenga et al. (2015) </font>

In [8]:
for i in range(df_falenga.shape[0]):
    # Object id
    id = df_falenga.loc[i, 'id']
    # Spectral type
    spectral_type = df_hmxb[df_hmxb["id"] == str(id)]["ST"].reset_index(drop=True).at[0]

    # Put R in luminosity dataframe
    df_R.loc[df_R["id"] == id, "R_true"] = df_falenga['R_opt'][i]
    df_R.loc[df_R["id"] == id, "R_true_err"] = df_falenga["R_opt_err"][i]
    df_R.loc[df_R["id"] == id, "ST"] = spectral_type

In [9]:
# Calculate ratio
df_R["R_expected/R_true"] = df_R["R_expected"] / df_R["R_true"]

In [10]:
df_R = df_R.dropna(subset=['R_true', 'R_expected']).reset_index(drop=True)

<font color='yellow' size=5>Mass </font>
- M_HRD is determined from the position in the HRD as shown below in the code
- M_opt is the determined mass from Falenga et al. (2015) based on the orbital parameters

In [11]:
df_M = pd.read_excel("../tables/M_expBONNSAI.xlsx")
df_M = pd.merge(df_M, df_falenga, on='id')
# [["id", "M_HRD", "M_opt", "M_opt_err", "M_exp", "M_exp_err"]]
df_M["M_opt/M_exp"] = df_M["M_opt"] / df_M["M_exp"]

<font color='yellow' size=5>All parameters</font>

Calculate errors on M/M and R/R

In [12]:
df_M["M_opt/M_exp_err"] = ( (df_M["M_opt_err"] / df_M["M_exp"])**2 + (df_M["M_opt"] / (df_M["M_exp"])**2 * df_M["M_exp_err"] )**2 )**(1/2)
df_R["R_expected/R_true_err_high"] = ( (df_R["R_expected_err_high"] / df_R["R_true"])**2 + (df_R["R_expected"] / (df_R["R_true"])**2 * df_R["R_true_err"])**2 )**(1/2)
df_R["R_expected/R_true_err_low"] = ( (df_R["R_expected_err_low"] / df_R["R_true"])**2 + (df_R["R_expected"] / (df_R["R_true"])**2 * df_R["R_true_err"])**2 )**(1/2)

In [13]:
df_MRL = pd.merge(df_M, pd.merge(df_R, df_L, on='id'), on='id').drop(columns=['ST_y']).rename(columns={"ST_x": 'ST'})
df_MRL = df_MRL.drop(df_MRL.loc[df_MRL['id'] == 'OAO 1657-415'].index)

In [14]:
df_MRL['id']

0          SMC X-1
1         Vela X-1
2          LMC X-4
3        4U1700-37
4        4U1538-52
5          Cen X-3
6    XTE J1855-026
Name: id, dtype: object

<font color='yellow' size=5>Roche-lobe radius</font>

In [16]:
# Gather all data needed to calculate roche lobe radius
df_falenga7 = falenga_table7()
df_roche_lobe = pd.DataFrame()
df_roche_lobe['id'] = df_MRL['id'].copy()
df_roche_lobe = pd.merge(df_roche_lobe, df_falenga7, on='id')
df_roche_lobe = pd.merge(df_roche_lobe, df_MRL)

Calculate RL based on falenga data

In [17]:
# Check for non-numeric values and convert to NaN
df_roche_lobe[['a', 'RL/a', 'RL/a_err', 'a_err', 'Mopt', 'Mopt_err', 'Mns', 'Mns_err', 'Ropt', 'Ropt_err']] = df_roche_lobe[['a', 'RL/a', 'RL/a_err', 'a_err', 'Mopt', 'Mopt_err', 'Mns', 'Mns_err', 'Ropt', 'Ropt_err']].apply(pd.to_numeric, errors='coerce')

# Perform the multiplication
df_roche_lobe['RL_fal'] = df_roche_lobe['RL/a'] * df_roche_lobe['a']
# Perform the calculation
df_roche_lobe['RL_fal_err'] = np.sqrt((df_roche_lobe['a'] * df_roche_lobe['RL/a_err']) ** 2 + (df_roche_lobe['RL/a'] * df_roche_lobe['a_err']) ** 2)

Calculate RL based on q function

In [18]:
df_roche_lobe['RL_q'] = np.nan
df_roche_lobe['RL_q_err'] = np.nan

for object in df_roche_lobe['id']:
    # Gather parameters
    a = df_roche_lobe.loc[df_roche_lobe['id'] == object, 'a'].reset_index(drop=True)[0]
    a_err = df_roche_lobe.loc[df_roche_lobe['id'] == object, 'a_err'].reset_index(drop=True)[0]
    M1 = df_roche_lobe.loc[df_roche_lobe['id'] == object, 'Mopt'].reset_index(drop=True)[0]
    M1_err = df_roche_lobe.loc[df_roche_lobe['id'] == object, 'Mopt_err'].reset_index(drop=True)[0]
    M2 = df_roche_lobe.loc[df_roche_lobe['id'] == object, 'Mns'].reset_index(drop=True)[0]
    M2_err = df_roche_lobe.loc[df_roche_lobe['id'] == object, 'Mns_err'].reset_index(drop=True)[0]
    
    RL, RL_err = roche_lobe_radius_q(a, a_err, M1, M1_err, M2, M2_err)
    df_roche_lobe.loc[df_roche_lobe['id'] == object, 'RL_q'] = RL
    df_roche_lobe.loc[df_roche_lobe['id'] == object, 'RL_q_err'] = RL_err

Calculate RL based on g function

In [19]:
df_roche_lobe['RL_g'] = np.nan
df_roche_lobe['RL_g_err'] = np.nan

for object in df_roche_lobe['id']:
    # Gather the variables
    a = df_roche_lobe.loc[df_roche_lobe['id'] == object, 'a'].reset_index(drop=True)[0]
    a_err = df_roche_lobe.loc[df_roche_lobe['id'] == object, 'a_err'].reset_index(drop=True)[0]
    M = df_roche_lobe.loc[df_roche_lobe['id'] == object, 'Mns'].reset_index(drop=True)[0]
    M_err = df_roche_lobe.loc[df_roche_lobe['id'] == object, 'Mns_err'].reset_index(drop=True)[0]
    R = df_roche_lobe.loc[df_roche_lobe['id'] == object, 'Mopt'].reset_index(drop=True)[0]
    R_err = df_roche_lobe.loc[df_roche_lobe['id'] == object, 'Mopt_err'].reset_index(drop=True)[0]

    # Calculate RL + err
    RL, RL_err = roche_lobe_radius_g(a, a_err, R, R_err, M, M_err)

    # Save
    df_roche_lobe.loc[df_roche_lobe['id'] == object, 'RL_g'] = RL
    df_roche_lobe.loc[df_roche_lobe['id'] == object, 'RL_g_err'] = RL_err

Devide RL  by Rtrue and calculate error

In [20]:
# Calculate the fraction for Falenga RL
df_roche_lobe['RL_fal/R_true'] = df_roche_lobe['RL_fal'] / df_roche_lobe['R_true']
df_roche_lobe['RL_fal/R_true_err'] = ( (df_roche_lobe["RL_fal_err"] / df_roche_lobe["R_true"])**2 + (df_roche_lobe["RL_fal"] / (df_roche_lobe["R_true"])**2 * df_roche_lobe["R_true_err"] )**2 )**(1/2)

# Also the inverse fraction
df_roche_lobe['R_true/RL_fal'] = df_roche_lobe['R_true'] / df_roche_lobe['RL_fal']
df_roche_lobe['R_true/RL_fal_err'] = ( (df_roche_lobe["R_true_err"] / df_roche_lobe["RL_fal"])**2 + (df_roche_lobe["R_true"] / (df_roche_lobe["RL_fal"])**2 * df_roche_lobe["RL_fal_err"] )**2 )**(1/2)

# Calculate the fraction with the RL based on q
df_roche_lobe['RL_q/R_true'] = df_roche_lobe['RL_q'] / df_roche_lobe['R_true']
df_roche_lobe['RL_q/R_true_err'] = ( (df_roche_lobe["RL_q_err"] / df_roche_lobe["R_true"])**2 + (df_roche_lobe["RL_q"] / (df_roche_lobe["R_true"])**2 * df_roche_lobe["R_true_err"] )**2 )**(1/2)

# Calculate the fraction with the RL based on g
df_roche_lobe['RL_g/R_true'] = df_roche_lobe['RL_g'] / df_roche_lobe['R_true']
df_roche_lobe['RL_g/R_true_err'] = ( (df_roche_lobe["RL_g_err"] / df_roche_lobe["R_true"])**2 + (df_roche_lobe["RL_g"] / (df_roche_lobe["R_true"])**2 * df_roche_lobe["R_true_err"] )**2 )**(1/2)

In [21]:
df_roche_lobe_cut = df_roche_lobe[['id', 'RL_fal', 'RL_fal_err', 'RL_q', 'RL_q_err', 'RL_g', 'RL_g_err', 'RL_fal/R_true', 'RL_fal/R_true_err', 'RL_q/R_true', 
                                   'RL_q/R_true_err', 'RL_g/R_true', 'RL_g/R_true_err', 'M_opt/M_exp', 'M_opt/M_exp_err', 'R_expected/R_true', 'R_expected/R_true_err_low',
                                   'R_true/RL_fal', 'R_true/RL_fal_err']]
# df_roche_lobe_cut.to_csv('../tables/roche_lobe.csv', sep=',')

<font color='yellow' size=5>X-ray luminosity</font>

In [23]:
df_xRayLum_clean = pd.read_excel('../tables/X_ray_lum.xlsx', header=0, na_values=None)
df_xRayLum = pd.merge(df_xRayLum_clean, df_BJ[['id', 'r_med_photogeo']], on='id')
df_xRayLum = pd.merge(df_xRayLum, df_falenga[['id', 'P_spin']], on='id')
df_xRayLum['L_x'] = 4 * np.pi * (df_xRayLum['r_med_photogeo'] * 3.086E18)**2 * df_xRayLum['X_rayFlux_SWIFTXRT'] * 10E-12

,id,X_rayFlux_SWIFTXRT,SWIFTXRT_range,SWIFT_id,r_med_photogeo,P_spin,L_x
0,Vela X-1,1759.0,0.3-10.0 keV,NaN,1969.4784,283.20,8.165259e+36
1,Cen X-3,3861.0,0.3-10.0 keV,NaN,6752.4946,4.80,2.106832e+38
2,4U1538-52,271.9,0.3-10.0 keV,NaN,5535.3643,526.80,9.970188e+36
3,4U1700-37,1157.0,0.3-10.0 keV,NaN,1513.9932,NaN,3.173825e+36
4,XTE J1855-026,218.7,0.3-10.0 keV,NaN,7386.8710,360.70,1.428142e+37
5,SMC X-1,992.4,0.3-10.0 keV,J0117.8-7327,62440.0000,0.71,4.630355e+39
6,LMC X-4,0.0,-,J0532.5-6623A,48590.0000,13.50,0.000000e+00


In [24]:
df_xRayLum_RL = pd.merge(df_xRayLum, df_roche_lobe[['id', 'R_expected', 'R_true', 'RL_fal', 'RL_q', 'RL_g', 'R_true/RL_fal', 'R_true/RL_fal_err']], on='id')

<font color='yellow' size=5>Merge all data to 1 DataFrame</font>

In [25]:
df_Complete = pd.merge(df_hmxb, pd.merge(df_falenga, df_BJ, on='id'), on='id')

df_Complete = pd.merge(df_Complete, df_L, on=['id', 'ST'])

df_Complete = pd.merge(df_Complete, df_R, on=['id', 'ST'])


df_Complete = pd.merge(df_Complete, df_M, on=['id', 'semi_eclipse_angle', 'semi_eclipse_angle _err', 'a', 'a_err', 'i', 'i_err', 'M_opt', 'M_opt_err', 
                                              'R_opt', 'R_opt_err', 'P_orbit', 'P_spin', 'RL/a', 'RL/a_err', 'Mx', 'Mx_err'])

df_Complete = pd.merge(df_Complete, df_roche_lobe, on=['id', 'ST', 'semi_eclipse_angle', 'semi_eclipse_angle _err', 'a', 'a_err', 'i', 'i_err', 'M_opt', 'M_opt_err', 
                                                       'R_opt', 'R_opt_err', 'P_orbit', 'P_spin', 'RL/a', 'RL/a_err', 'Mx', 'Mx_err', 'L_true', 'L_true_err_low', 
                                                       'L_true_err_high', 'logL_true/L_sun', 'logL_true/L_sun_err_high', 'logL_true/L_sun_err_low', 'Teff', 
                                                       'Teff_err', 'logTeff', 'logTeff_err', 'R_true', 'R_true_err', 'R_expected', 'R_expected_err_high', 
                                                       'R_expected_err_low', 'R_expected/R_true', 'M_exp', 'M_HRD', 'M_exp_err', 'M_opt/M_exp'])

df_Complete = pd.merge(df_Complete, df_xRayLum, on=['id', 'P_spin', 'r_med_photogeo'])


In [27]:
df_Complete.to_csv('../tables/results/ALLPARAMETERS.csv')